In [8]:
import tensorflow as tf
# from tensorflow.python.keras import Model, layers
from keras import Model, layers
import os
from PIL import Image
import numpy as np

In [9]:
class SmpDataset(tf.keras.utils.Sequence):
    
    def __init__(self, root_dir, batch_size=32, shuffle=True):
        self.root_dir = root_dir
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.images = []
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                self.images.append((os.path.join(cls_dir, img_name), self.class_to_idx[cls_name]))

        if self.shuffle:
            np.random.shuffle(self.images)

    def __len__(self):
        return len(self.images) // self.batch_size

    def __getitem__(self, index):
        batch_images = self.images[index * self.batch_size:(index + 1) * self.batch_size]
        X = []
        y = []
        for img_path, label in batch_images:
            image = Image.open(img_path).convert('RGB')
            image = np.array(image.resize((64, 64))) / 255.0  # Normalize to [0, 1]
            X.append(image)
            y.append(label)

        return np.array(X), np.array(y)
    


In [10]:
class NN(Model):
    def __init__(self, num_classes):
        super(NN, self).__init__()
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512, activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.fc2 = layers.Dense(256, activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.fc3 = layers.Dense(num_classes)

    def call(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        return self.fc3(x)

In [11]:

# Device configuration
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_classes = 2
learning_rate = 0.0001
batch_size = 32
num_epochs = 200
# Load dataset
train_data = SmpDataset(root_dir='./homer_bart/', batch_size=batch_size)

model = NN(num_classes=num_classes)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


for epoch in range(num_epochs):
    for step in range(len(train_data)):
        batch_x, batch_y = train_data[step]
        
        with tf.GradientTape() as tape:
            logits = model(batch_x, training=True)
            loss_value = loss_object(batch_y, logits)

        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss_value.numpy():.4f}")


Epoch [0/200], Loss: 0.6645
Epoch [10/200], Loss: 0.4954
Epoch [20/200], Loss: 0.3067
Epoch [30/200], Loss: 0.1599
Epoch [40/200], Loss: 0.0833
Epoch [50/200], Loss: 0.0410
Epoch [60/200], Loss: 0.0447
Epoch [70/200], Loss: 0.0859
Epoch [80/200], Loss: 0.2679
Epoch [90/200], Loss: 0.0790
Epoch [100/200], Loss: 0.0353
Epoch [110/200], Loss: 0.0217
Epoch [120/200], Loss: 0.0143
Epoch [130/200], Loss: 0.0103
Epoch [140/200], Loss: 0.0079
Epoch [150/200], Loss: 0.0060
Epoch [160/200], Loss: 0.0049
Epoch [170/200], Loss: 0.0041
Epoch [180/200], Loss: 0.0034
Epoch [190/200], Loss: 0.0029


In [12]:
def check_accuracy(data, model):
    num_correct = 0
    num_samples = 0
    for step in range(len(data)):
        batch_x, batch_y = data[step]
        logits = model(batch_x)
        predictions = tf.argmax(logits, axis=1)
        num_correct += np.sum(predictions.numpy() == batch_y)
        num_samples += batch_y.shape[0]

    accuracy = num_correct / num_samples * 100
    print(f'Accuracy: {accuracy:.2f}%')

check_accuracy(train_data, model)

Accuracy: 100.00%
